# Exploration and Analysis
- <Description goes here>

In [ ]:
# Dependencies and Imports
import pandas as pd
import requests
import json
import matplotlib as mlt
import gmaps
from census import Census
from us import states
import numpy as np

## Need to move to config file
# Census API Key
census_API = 'ad9e020616792d8cbe0f666089dc3980b9d379e0'
c=Census(census_API)

# Import and preview the dataset in a dataframe
path = './Resources/311_Pothole_History.csv'
df = pd.DataFrame(pd.read_csv(path))

# Preview the dataset
print(f"Total # of complaints in the dataset: {len(df)}")
df.head()

In [2]:
# Investigating values for non-standard columns
print(f"Source values: {df['SOURCE'].unique()}")
print(f"Work group values: {df['WORK GROUP'].unique()}")
print(f"Request type values: {df['REQUEST TYPE'].unique()}")
print(f"Status values: {df['STATUS'].unique()}")

Source values: ['PHONE' 'WEB' 'EMAIL' 'BOT' 'TWIR' 'WALK' 'EDC' 'VOICE' 'FAX' 'INSPE'
 'SYS' 'MAIL' 'EIP' 'SPNSH']
Work group values: ['Public Works-Street and Traffic-District 3'
 'Public Works-Street and Traffic-District 1'
 'Public Works-Street and Traffic-District 2'
 'City Managers Office-311 Call Center-Support'
 'Public Works-Capital Projects-Sidewalks' 'Parks and Rec-South Region-'
 'Public Works-Capital Projects-Traffic Permits'
 'Parks and Rec-Central Region-' 'Parks and Rec-Administration-']
Request type values: ['Streets / Roadways / Alleys-Pothole-District 3'
 'Streets / Roadways / Alleys-Pothole-District 1'
 'Streets / Roadways / Alleys-Pothole-District 2'
 'Streets / Roadways / Alleys-Pothole-Bridge' 'Pothole (North of River)'
 'Pothole (South of 47th Street and West Of Blue PKWY)'
 'Pothole (South of 47th Street)'
 'Pothole (South of River to 47th Street)'
 'Pothole (River south to 47th Street)'
 'Pothole (River south to 47th Street and East of Blue PKWY)'
 'Streets / R

In [3]:
# Identifying data ranges
print(f"Dataset covers {df['CREATION YEAR'].nunique()} years, {df['ZIP CODE'].nunique()} zipcodes, and {df['NEIGHBORHOOD'].nunique()} neighborhoods")

Dataset covers 14 years, 55 zipcodes, and 249 neighborhoods


In [4]:
# Begin cleaning data. We'll start by only using resolved complaints (open/canceled complaints lack some of the data we'd need if we want to use closing dates)
resolved_df = df.loc[(df['STATUS']=='RESOL')]
print(f"Current number of rows: {len(resolved_df)}")

# Cut down the number of columns to main identifying data. We'll primarily be using zip codes to pull census data
# Extra location data and "closed" data has been removed for ease.
reduced_resolved_df = resolved_df[['CASE ID', 'SOURCE', 'CREATION MONTH', 
                                   'CREATION YEAR', 'STATUS', 'EXCEEDED EST TIMEFRAME', 
                                   'DAYS TO CLOSE', 'ZIP CODE', 'NEIGHBORHOOD']]

# Drop any rows that still have missing data
clean_df = reduced_resolved_df.dropna(how='any').copy()
clean_df.reset_index(inplace=True)
print(f"Clean number of rows: {len(clean_df)}")

# Format columns to drop decimal places
clean_df['ZIP CODE'] = clean_df['ZIP CODE'].astype(int)
clean_df['DAYS TO CLOSE'] = clean_df['DAYS TO CLOSE'].astype(int)

# Output the dataframe to a csv and preview the final cleaned dataframe
clean_df.to_csv('./Resources/PotholeData.csv', index=False)
clean_df

Current number of rows: 49499
Clean number of rows: 48070


,index,CASE ID,SOURCE,CREATION MONTH,CREATION YEAR,STATUS,EXCEEDED EST TIMEFRAME,DAYS TO CLOSE,ZIP CODE,NEIGHBORHOOD
0,0,2019169222,PHONE,9,2019,RESOL,Y,48,64133,Blue Vue Hills
1,1,2019126105,PHONE,7,2019,RESOL,Y,113,64131,East Meyer 6
2,2,2019126468,WEB,7,2019,RESOL,Y,113,64130,Blue Hills
3,3,2019183619,PHONE,10,2019,RESOL,Y,30,64119,Shoal Creek
4,4,2019190590,PHONE,11,2019,RESOL,Y,10,64134,Fairlane
...,...,...,...,...,...,...,...,...,...,...
48065,50068,2019170484,PHONE,9,2019,RESOL,N,1,64126,East Blue Valley
48066,50069,2019170942,PHONE,9,2019,RESOL,Y,52,64110,Western 49-63
48067,50070,2019171791,PHONE,9,2019,RESOL,Y,50,64134,Hickman Mills South
48068,50071,2019170397,TWIR,9,2019,RESOL,N,2,64108,Wendell Phillips


In [5]:
# Optional reduced dataframe for fewer years of data (different census data pull)
fifteen_nineteen_df = clean_df[clean_df["CREATION YEAR"] > 2014]
fifteen_nineteen_df.reset_index(inplace=True)
fifteen_nineteen_df

,level_0,index,CASE ID,SOURCE,CREATION MONTH,CREATION YEAR,STATUS,EXCEEDED EST TIMEFRAME,DAYS TO CLOSE,ZIP CODE,NEIGHBORHOOD
0,0,0,2019169222,PHONE,9,2019,RESOL,Y,48,64133,Blue Vue Hills
1,1,1,2019126105,PHONE,7,2019,RESOL,Y,113,64131,East Meyer 6
2,2,2,2019126468,WEB,7,2019,RESOL,Y,113,64130,Blue Hills
3,3,3,2019183619,PHONE,10,2019,RESOL,Y,30,64119,Shoal Creek
4,4,4,2019190590,PHONE,11,2019,RESOL,Y,10,64134,Fairlane
...,...,...,...,...,...,...,...,...,...,...,...
37124,48065,50068,2019170484,PHONE,9,2019,RESOL,N,1,64126,East Blue Valley
37125,48066,50069,2019170942,PHONE,9,2019,RESOL,Y,52,64110,Western 49-63
37126,48067,50070,2019171791,PHONE,9,2019,RESOL,Y,50,64134,Hickman Mills South
37127,48068,50071,2019170397,TWIR,9,2019,RESOL,N,2,64108,Wendell Phillips


In [6]:
# Assembling a clean list of the zip codes in the data set
zip_codes = df['ZIP CODE'].dropna().unique()
zip_codes = zip_codes.astype(int)
zip_codes

array([64133, 64131, 64130, 64119, 64134, 64108, 64113, 64132, 64112,
       64146, 64127, 64125, 64149, 64138, 64114, 64145, 64105, 64123,
       64110, 64137, 64155, 64109, 64154, 64151, 64124, 64116, 64120,
       64139, 64152, 64129, 64158, 64106, 64111, 64128, 64118, 64117,
       64136, 64126, 64157, 64156, 64101, 64163, 64153, 64161, 64147,
       64167, 64102, 64166, 64165, 64164, 64160, 64068, 64012, 64052,
       64030])

In [ ]:
# generate an empty list to fill with the census data
data = []

# iterate through the list of zip codes to pull the census data
for ea in zip_codes:
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B19301_001E", "B17001_002E"), {'for': 'zip code tabulation area:'+str(ea)})
    try:
        data.append(census_data[0])
    except:
        next

In [19]:
# Store the data into a dataframe
data_df = pd.DataFrame(data)
data_df

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,zip code tabulation area
0,ZCTA5 64133,50681.0,35163.0,40.8,26720.0,4690.0,64133
1,ZCTA5 64131,45688.0,22932.0,37.2,30411.0,4021.0,64131
2,ZCTA5 64130,31166.0,19996.0,39.3,21925.0,5238.0,64130
3,ZCTA5 64119,68841.0,27848.0,38.3,33224.0,1785.0,64119
4,ZCTA5 64134,41543.0,23874.0,31.3,20078.0,5507.0,64134
5,ZCTA5 64108,49500.0,7799.0,35.4,39759.0,1590.0,64108
6,ZCTA5 64113,136913.0,11988.0,42.5,73133.0,248.0,64113
7,ZCTA5 64132,36074.0,14817.0,31.6,18374.0,4542.0,64132
8,ZCTA5 64112,74864.0,8352.0,35.7,78107.0,769.0,64112
9,ZCTA5 64146,60667.0,1506.0,42.3,34966.0,93.0,64146


In [20]:
# Renaming the columns of the dataframe
data_df = data_df.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Reordering the columns, dropping the "Name" column
data_df = data_df[['Zipcode','Household Income','Population','Median Age', 'Per Capita Income', 'Poverty Count']]

In [22]:
# Removing oddball data from the dataset
data_df = data_df.loc[(data_df['Population']> 0) & (data_df['Household Income'] > 0)]
data.head()

[35163. 22932. 19996. 27848. 23874.  7799. 11988. 14817.  8352.  1506.
 15457.  1389.   355. 26032. 24405.  5249.  4679.  9580. 17176. 10646.
 22917.  9647. 10895. 26777. 10626. 15576.   383.  2079. 29231.  8881.
  5041.  9326. 17801. 11379. 42706. 14673.  2220.  6148. 19918.  7935.
   278.   748.  5234.   300.   581.   446.     0.   285.   215.   213.
 38439. 27190. 20297. 25150.]
[ 5.06810000e+04  4.56880000e+04  3.11660000e+04  6.88410000e+04
  4.15430000e+04  4.95000000e+04  1.36913000e+05  3.60740000e+04
  7.48640000e+04  6.06670000e+04  2.54720000e+04  2.23280000e+04
  1.16719000e+05  4.69150000e+04  6.36940000e+04  7.16110000e+04
  5.94740000e+04  3.52900000e+04  4.25500000e+04  4.73240000e+04
  8.46140000e+04  3.21310000e+04  7.34580000e+04  7.11900000e+04
  2.65120000e+04  4.93520000e+04  3.44170000e+04  1.13553000e+05
  8.89190000e+04  3.93960000e+04  8.27390000e+04  3.11400000e+04
  4.81050000e+04  2.44210000e+04  5.56900000e+04  4.65140000e+04
  3.65380000e+04  2.06990000e+

In [ ]:
data_df.to_csv('./Resources/CensusData.csv', index=False)

In [ ]:
ZipCodes = list(data_df['Zipcode'])
ZipCodes = pd.DataFrame(ZipCodes)
ZipCodes.to_c